In [3]:
import numpy as np
import pandas as pd
import dlib
import cv2
from imutils import face_utils
from main import HeadPoseEstimator
from utils import DataCleaner, split_train_dev_x_y, draw_axis

In [6]:
np.random.seed(42)

df_path = '../cleaned_data_68pts.csv'
df = pd.read_csv(df_path, index_col=0)

X_train, Y_train, X_dev, Y_dev = split_train_dev_x_y(df, train_size=0.99)

model = HeadPoseEstimator()
model = model.fit(X_train, Y_train)
# model.report(X_train, Y_train, X_dev, Y_dev)


In [9]:
# Let's go life
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/home/ahmad/projects/python/head_pose_estimation/data')

cap = cv2.VideoCapture(0)
# points = {17, 19, 21, 22, 24, 26, 36, 39, 42, 45, 31, 30, 35, 48, 66, 54, 8, 2, 14}

while True:
    success, image = cap.read()

    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # To improve performance
    # image.flags.writeable = False
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Get the result
    rects = detector(image, 1)
    # loop over the face detections
    for rect in rects:
        
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(image, rect)
        shape = face_utils.shape_to_np(shape)
        nose = shape[30]
        features = DataCleaner()._get_features(shape)
        features = np.delete(features, [30, 98], axis=1)
        # pitch, yaw, roll = model.predict(features)
        # draw_axis(image, pitch, yaw, roll, nose)
        # for n in points: # range(1, 68):
        #     x = shape.part(n).x
        #     y = shape.part(n).y
        #     cv2.circle(image, (x, y), 1, (0, 255, 255), 1)

    
        
    # To improve performance
    # image.flags.writeable = True

    # Convert the color space from RGB to BGR
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imshow("Face Landmarks", image)

    key = 0xFF & cv2.waitKey(1)
    if key & 0xFF == 27: #Exit program when the user presses 'esc'
        break
    
cv2.destroyAllWindows()
cap.release()



In [19]:
cap = cv2.VideoCapture('/home/ahmad/Documents/video.mp4')
# Obtain frame size information using get() method
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width,frame_height)
fps = 20
output = cv2.VideoWriter(
    '/home/ahmad/Documents/video_mod.mp4', 
    cv2.VideoWriter_fourcc(*'XVID'), 
    20, 
    frame_size) 
    
# Loop until the end of the video
i = 0
while cap.isOpened() and i < 300:
    i += 1
    success, image = cap.read()
    # Display the resulting frame
    # cv2.imshow('Frame', image)
    rects = detector(image, 1)
    # loop over the face detections
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(image, rect)
        shape = face_utils.shape_to_np(shape)
        nose = shape[30]
        features = DataCleaner()._get_features(shape)
        features = np.delete(features, [30, 98], axis=1)
        pitch, yaw, roll = model.predict(features)
        draw_axis(image, pitch, yaw, roll, nose)
        output.write(image)

    
    # Display the resulting frame
    # cv2.imshow('Frame', image)
 
 
    key = 0xFF & cv2.waitKey(1)
    if key & 0xFF == 27: #Exit program when the user presses 'esc'
        break
    
cv2.destroyAllWindows()
cap.release()
output.release()


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
